In [1]:
%pwd

'n:\\DataScience\\MyProjects\\DeepLearning-Project-With-MLFlow-DVC-Deployment\\research'

In [2]:
import os
os.chdir('../')
%pwd

'n:\\DataScience\\MyProjects\\DeepLearning-Project-With-MLFlow-DVC-Deployment'

### Update the entity

In [3]:
# import required libraries
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

### Update the configuration manager in src config

In [4]:
from DLProject.constants import *
from DLProject.utils.common import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

### Update the components

In [6]:
import os
import zipfile
import gdown
from DLProject import logger
from DLProject.utils.common import get_size

In [7]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        dataset_url = self.config.source_URL
        zip_download_dir = self.config.local_data_file
        root_download_dir = self.config.root_dir

        os.makedirs(root_download_dir, exist_ok=True)
        logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

        FILE_ID = dataset_url.split("/")[-2]
        PREFIX_URL = "https://drive.google.com/uc?/export=download&id="

        DOWNLOAD_URL = f'{PREFIX_URL}{FILE_ID}'
        try:
            gdown.download(DOWNLOAD_URL,  zip_download_dir)
            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            logger.error(f"Error downloading data from {dataset_url}: {e}")
            raise ValueError(f"Failed to download data from {dataset_url}")

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory.

        Returns:
            None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)

        zip_file_path = self.config.local_data_file

        if not os.path.exists(zip_file_path):
            raise FileNotFoundError(f"Zip file not found at {zip_file_path}")
        try:
            with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
                zip_ref.extractall(unzip_path)
                logger.info(f"Extracted zip file to {unzip_path}")
                
        except zipfile.BadZipFile as e:
            logger.error(f"Error extracting zip file: {e}")
            raise ValueError("Failed to extract zip file")

### Update the pipeline

In [8]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2023-10-10 14:35:43,217: INFO: common: YAML file: config\config.yaml loaded successfully]
[2023-10-10 14:35:43,230: INFO: common: YAML file: params.yaml loaded successfully]
[2023-10-10 14:35:43,237: INFO: common: Created directory at: artifacts]
[2023-10-10 14:35:43,269: INFO: common: Created directory at: artifacts/data_ingestion]
[2023-10-10 14:35:43,279: INFO: 3505116073: Downloading data from https://drive.google.com/file/d/17bCzkEA2IdlnW8T1HoFTPdBAxhx9r4kk/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (uriginal): https://drive.google.com/uc?/export=download&id=17bCzkEA2IdlnW8T1HoFTPdBAxhx9r4kk
From (redirected): https://drive.google.com/uc?/export=download&id=17bCzkEA2IdlnW8T1HoFTPdBAxhx9r4kk&confirm=t&uuid=ae305d67-ccb8-49b2-82db-58049c7e8d29
To: n:\DataScience\MyProjects\DeepLearning-Project-With-MLFlow-DVC-Deployment\artifacts\data_ingestion\data.zip
100%|██████████| 1.63G/1.63G [05:57<00:00, 4.55MB/s]  

[2023-10-10 14:41:43,543: INFO: 3505116073: Downloaded data from https://drive.google.com/file/d/17bCzkEA2IdlnW8T1HoFTPdBAxhx9r4kk/view?usp=sharing into file artifacts/data_ingestion/data.zip]


[2023-10-10 15:03:45,133: INFO: 3505116073: Extracted zip file to artifacts/data_ingestion]
